In [7]:
include("StackyFan.jl")

coneRayDecomposition (generic function with 1 method)

In [8]:
"""

    BerghA(F::StackyFan,D::Array{Int64,1})

    Given a stacky fan F and a vector of booleans D representing the distinguished structure,
    returns a smooth stacky fan where the distinguished rays are independent.

    
"""

function BerghA(F::StackyFan,D::Array{Int64,1},verbose::Bool=true)
    if verbose==true
        println("==algorithm is running in verbose mode==")
        println(" ")
        println("=======")
    end
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
    coneList=getCones(X.fan)
    dim=size(rayMatrix,2)
    numRays=size(rayMatrix,1)
    stackTracker=ones(Int64,numRays)
    
    #check if the vector D has length equal to the number of rays in F
    if numRays != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0: initialization
    while(true)
        rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        numRays=size(rayMatrix,1)
        coneList=getCones(X.fan)
        if verbose==true
            #println("Ray matrix: $rayMatrix")
            #println("Cone list: $coneList")
            #sW=stackyWeights(X)
            #println("Stacky weights: $sW")
            #println("Distinguished rays: $D")
        end
        coneMultiplicities=Int64[]
        for cone in coneList
            C=coneConvert(cone,rayMatrix)
            push!(coneMultiplicities,coneMultiplicity(C))
        end
        #if verbose==true
            #println("Cone multiplicities: $coneMultiplicities")
        #end
        #A1: check if finished
        # Find S the set of cones that contain a distinguised ray and an interior a lattice point 
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndIntPoint(cone,rayMatrix,D),coneList)
        # If S is empty, the program terminates.
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        if verbose==true
            Smaxcount=size(Smax,1)
            println("Number of extremal cones: $Smaxcount")
            testCone=Smax[1]
            c1=convertToIncidence(testCone,numRays)
            nonDist=size(testCone,1)-dot(c1,D)
            mult=coneMultiplicity(coneConvert(testCone,rayMatrix))
            println("Maximal non-distinguished rays and multiplicity: $nonDist, $mult")
        end
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            #C=rowMinors(rayMatrix,cone)
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            if coneIntPoints==nothing
                # does the program ever reach this line if it breaks at S empty?
                return C
            end
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=Array{Int64,1}[]
        numatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        for (point,cone) in intPoints
            stackyPoint=coneRayDecomposition(cone,rayMatrix,point,stackyWeights(X))
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        psi=minimalByLex(P)
        if verbose==true
            #println("Psi: $psi")
        end
        
        
        #A3 - perform root construction
        X=rootConstructionDistinguishedIndices(X,D,psi)
        
        #A3 - modify psi with respect to root construction
        for i in 1:length(psi)
            if D[i]==1 && psi[i]>0
                psi[i]=1
            end
        end
        
        #TODO: Currently the distinguished rays D and psi coefficients are not
        #used in the output. Should these be removed? Also may need to annotate
        #what stackTracker does.
        
        # Convert arrays to dictionaries as constructing fan objects through subdivision
        # may reorder rays
        
        
        #A5 - repeat star subdivision
        while(count(x->x>0,psi)>1)
            #A4 - perform stacky star subdivision
            # Get the indices of the non-zero coefficients in psi
            supportCone=findall(x->x!=0,psi)
            #if verbose==true
                #sW=stackyWeights(X)
                #println("Modified stacky weights: $sW")
            #end
            exceptional=findStackyBarycenter(supportCone,X)
            #if verbose==true
                #println("Blowing up at $exceptional")
            #end
            
            code_rays = mapslices(encode, Polymake.common.primitive(X.fan.RAYS), dims=2)
            # Track the indices of distinguished rays
            D_pairs = map((x,y) -> (x,y), code_rays, D)
            D_Dict = Dict(D_pairs)
            # Track psi as a linear combination of the generators
            psiPairs = map((x,y) -> (x,y), code_rays,psi)
            psiDict = Dict(psiPairs)
            trackerPairs = map((x,y) -> (x,y), code_rays, stackTracker)
            trackerDict= Dict(trackerPairs)
            
            X=stackyBlowup(X,[x-1 for x in supportCone],exceptional)
            
            G=gcd(exceptional)
            primExcep=Polymake.common.primitive(exceptional)
            
            # Update the dictionaries storing fan information
            D_Dict[encode(primExcep)]=1
            psiDict[encode(primExcep)]=1
            trackerDict[encode(primExcep)]=G
            
            newRays=slicematrix(convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS))))
            newD=Int64[]
            newpsi=Int64[]
            newTracker=Int64[]
            for ray in newRays
                E=encode(ray)
                excepCode=encode(primExcep)
                push!(newD,D_Dict[E])
                #A4 - modify psi
                if E==excepCode
                    push!(newpsi,1)
                elseif psiDict[E]>1
                    push!(newpsi,psiDict[E]-1)
                else
                    push!(newpsi,0)
                end
                push!(newTracker,trackerDict[E])
            end
            psi=newpsi
            D=newD
            stackTracker=newTracker
            #A4 - modify psi
        end
        if verbose==true
            println("=======")
        end
    end
    
    return X, stackTracker
end

BerghA (generic function with 2 methods)

In [9]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 3; 5 2],INPUT_CONES=[[0,1]])
F=addStackStructure(X,[1,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x0000000011e388b0), Dict("1,3" => 1, "5,2" => 1))

In [10]:
C=BerghA(F,[0,1])

==algorithm is running in verbose mode==
 
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 13
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 5
Number of extremal cones: 1
Maximal non-distinguished rays and multiplicity: 1, 2
Number of extremal cones: 40
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 39
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 38
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 37
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 36
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 35
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 34
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 33
Maximal non-distinguished rays and multiplicity: 0, 65
Number of extremal cones: 32
Maximal n

Excessive output truncated after 524297 bytes.

 470351552; 492923744 470351552; 492923808 470351552; 492923872 470351552; 492923936 470351552; 492924000 470351552; 492924064 470351552; 492924128 470351552; 492924192 470351552; 492924256 470351552; 492924320 470351552; 492924384 470351552; 492924448 470351552; 492924512 470351552; 492924576 470351552; 492924640 470351552; 492924704 470351552; 492924768 470351552; 492924832 470351552; 492924896 470351552; 492924960 470351552; 492925024 470351552; 492925088 470351552; 492925152 470351552; 492925216 470351552; 492925280 470351552; 492925344 470351552; 492925408 470351552; 492925472 470351552; 492925536 470351552; 492925600 470351552; 492925664 470351552; 492925728 470351552; 492925792 470351552; 492925856 470351552; 492925920 492925952]
[492921760, 470351552]
ERROR!!!
KeyError("492921824,470351552")
[461234080 473784704; 461234144 492925984; 461234208 492925984; 461234272 492925984; 461234336 492925984; 470357712 492926048; 470357776 492925984; 470357840 492925984; 470357904 492925984;

LoadError: DimensionMismatch("matrix A has dimensions (0,0), matrix B has dimensions (191,2)")

In [54]:
interiorPoints(C)

Polymake.Rational[0 0; 1 3; 5 2; 6 5]


8-element Vector{Any}:
 Polymake.Integer[1, 1]
 Polymake.Integer[1, 2]
 Polymake.Integer[2, 1]
 Polymake.Integer[2, 3]
 Polymake.Integer[3, 2]
 Polymake.Integer[4, 3]
 Polymake.Integer[5, 3]
 Polymake.Integer[5, 4]